# Simulated Annealing


In [1]:
%matplotlib inline

# Pytorch libraries
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import random as rm
import numpy as np
import time
import copy
import math

# Add the sibling folders
import sys, os
sys.path.insert(0, os.path.abspath('../..'))
import src.utils as ut

# Plot libraries and tables
import matplotlib.pyplot as plt
import mpld3
mpld3.enable_notebook()

Let's define a basic NN, made out of two layers: the first one has 200 neurons, while the second one 50.

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Basic two-layer network
        self.fc1 = nn.Linear(28 * 28, 10)
        #self.fc2 = nn.Linear(200, 50)
        #self.fc3 = nn.Linear(50, 10)
        
    def forward(self, x):        
        # Get the batch size
        in_size = x.size(0)
        # Flatten data, -1 is inferred from the other dimensions
        x = x.view(in_size, -1) 
        
        # Forward rule
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        
        # Softmax on predictions
        #x = F.softmax(x, dim=1)
        
        return self.fc1(x)

In [2]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        #self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        #out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [4]:
class CifarNet(nn.Module):
    def __init__(self):
        super(CifarNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, bias=False)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5, bias=False)
        self.fc1 = nn.Linear(16 * 5 * 5, 10, bias=True)
        #self.fc2 = nn.Linear(120, 84, bias=False)
        #self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc1(x)
        #x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        return x

In [ ]:
net = ConvNet().cuda()
for name, param in net.named_parameters():
    print("Name:", name, " size: ", param.size())

train_loader, test_loader = ut.load_dataset(dataset_name='fashion-mnist', minibatch=512)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in range(10):
    print(ut.train(train_loader, net, optimizer, criterion, 1))
    print(ut.test(test_loader, net))

## The algorithm
The first implementation uses the **whole training set**, in order to avoid noisy evaluations. 
The epsilon variable allows to set the maximum movement in every direction. Each move is done by creating a tensor for each layer $k$ called $ \Delta_k(w)$. The update is done according to the formula:

$$ w'  = w - \epsilon \Delta(w)  $$

where each value of each tensor  $ \Delta_k(w) $ is sampled from the distribution $ U (-1, 1) $. 

In the following implementation I wanted to try a first test without making the best choice, in order to save some computation time. If $ L(w') > L(w) $ the opposite direction is chosen, i.e.

$$ w''  = w + \epsilon \ \Delta(w)  $$

even if $ L(w') < L(w'') $



In [ ]:
def full_train_SA_noT(trainloader, model, accuracy_before, epsilon, gpu=True):
    model.train()
    
    # List to store the opposite direction - \Delta(w) for each layer
    inverse = []
    
    for param in net.parameters():
        # Replicate the tensor
        tensor_size = param.data.size()
        move = torch.zeros(tensor_size)
        # Send it to the GPU
        if gpu:
            move = move.cuda()
        # Generate move
        move = move.uniform_(-1, 1).mul(epsilon) * param.data
        # Stepback is saved
        inverse.append(move.mul(-2))
        # Move the parameters
        param.data.add_(move)
    
    # Evaluate the accuracy 
    new_accuracy = ut.test_train(train_loader, net)[1]

    if new_accuracy < accuracy_before:
        for k, param in enumerate(net.parameters()):
            param.data.add_(inverse[k])

        new_accuracy = ut.test_train(train_loader, net)[1]
    
    #print("New accuracy: ", new_accuracy)
    return new_accuracy

Looking at the results (first plot below), it can be seen that both  directions do not often improve the accuracy. 

Now we can implement the full algorithm. $ L(w') $ and $ L(w'') $ are both evaulated:

$$ L_{best} = \min(L(w'),\ L(w'')) $$

If $ L_{best} < L(w) $, then accept the move for sure; otherwise accept if $ x  > temperature $, where x is sampled from the uniform distribution $ U(0, 1) $.


In [ ]:
def full_train_SA(trainloader, model, initial_accuracy, epsilon, T, gpu=True):
    model.train()
    
    # List used to keep the move to get back to the initial point
    inverse = []
    
    # First move
    for param in net.parameters():
        # Replicate the tensor
        tensor_size = param.data.size()
        move = torch.zeros(tensor_size)
        # Send it to the GPU
        if gpu:
            move = move.cuda()
        # Generate move
        move = move.uniform_(-1, 1).mul(epsilon) * param.data
        # Stepback is saved
        inverse.append(move.mul(-1)) 
        # Move the parameters
        param.data.add_(move)
    # Evaluate the accuracy 
    first_accuracy = ut.test_train(train_loader, net)[1]
    #print("First move accuracy: ", first_accuracy)
    
    # Second move
    for k, param in enumerate(net.parameters()):
        param.data.add_(inverse[k].mul(2))
        inverse[k] = inverse[k].mul(-1)
        print(param.size())
    second_accuracy = ut.test_train(train_loader, net)[1]
    #print("Second move accuracy:", second_accuracy)
    
    # Get back if the first accuracy is better
    if first_accuracy > second_accuracy:
        for k, param in enumerate(net.parameters()):
            param.data.add_(inverse[k].mul(2))
            inverse[k] = inverse[k].mul(-1)
        new_accuracy = first_accuracy
    else: new_accuracy = second_accuracy
    
    # Accept a worse solution according to temperature
    if new_accuracy < initial_accuracy and rm.uniform(0, 1) > T:
        for k, param in enumerate(net.parameters()):
            param.data.add_(inverse[k])
        new_accuracy = initial_accuracy
        
    del move, inverse
    #print("Final accuracy:", new_accuracy)
    return new_accuracy


Let's test the first method.

In [ ]:
# Parameters
epsilon = 10e-3
epochs = 2

In [ ]:
train_loader, test_loader = ut.load_dataset(dataset_name='mnist', minibatch=4096)

# Testing the first method
starting_time = time.time()
net = Net()
clone = copy.deepcopy(net)
net = net.cuda()
accuracy = ut.test_train(train_loader, net)[1]
training_set_measurements, validation_set_measurements, times = [], [], []

# Train
for epoch in range(epochs):
    #print("Epoch: ", epoch)
    accuracy = full_train_SA_noT(train_loader, net, accuracy, epsilon)
    training_set_measurements.append(accuracy)
    validation_set_measurements.append(ut.test(test_loader, net)[1])
    times.append(time.time() - starting_time)
    
np.savez('no_temperature',
            training_set_measurements=[x * 100 for x in training_set_measurements],
            validation_set_measurements=[x * 100 for x in validation_set_measurements],
            times=[x / 60 for x in times])
del net

Then the second method, forcing a monotonic behaviour, by alwasy setting $ Temperature = 0 $.

In [ ]:
# Testing the second method
net = clone.cuda()
accuracy = ut.test_train(train_loader, net)[1]
training_set_measurements, validation_set_measurements, times = [], [], []
starting_time = time.time()

for epoch in range(epochs):
    #print("Epoch: ", epoch)
    accuracy = full_train_SA(train_loader, net, accuracy, epsilon, 0)
    training_set_measurements.append(accuracy)
    validation_set_measurements.append(ut.test(test_loader, net)[1])
    times.append(time.time() - starting_time)
    
np.savez('T=0',
            training_set_measurements=[x * 100 for x in training_set_measurements],
            validation_set_measurements=[x * 100 for x in validation_set_measurements],
            times=[x / 60 for x in times])

del net

Two plots: the first one with 'number of epochs' as x-axis, the second one comparing the accuracy vs computing time.

In [ ]:
no_T = np.load('no_temperature.npz')
T_0 = np.load('T=0.npz')

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(np.arange(0, len(no_T['training_set_measurements'])), no_T['training_set_measurements'], '--', label='No T')
ax.plot(np.arange(0, len(T_0['training_set_measurements'])), T_0['training_set_measurements'], '--', label='T = 0')

plt.title('Comparison between the two implementations')
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy')
plt.title('')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(no_T['times'], no_T['training_set_measurements'], '--', label='No T')
ax.plot(T_0['times'], T_0['training_set_measurements'], '--', label='T = 0')

plt.xlabel('Minutes elapsed')
plt.ylabel('Accuracy')
ax.legend()
plt.show()

Some information about training and test acuracy in the following plot.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(np.arange(0, len(T_0['training_set_measurements'])), T_0['training_set_measurements'], '--', label='Training set')
ax.plot(np.arange(0, len(T_0['validation_set_measurements'])), T_0['validation_set_measurements'], '--', label='Validation set')

plt.title('Temperature = 0')
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy')
ax.legend()
plt.show()

In [3]:
def test_minibatch(images, labels, model):
    model.eval()
    correct = 0
    total = labels.size(0)
    test_loss = 0
    
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == labels).sum().item()
    test_loss += float(F.cross_entropy(outputs, labels).item())
    
    return test_loss / total, correct / total

In [4]:
def SGA(trainloader, model, epsilon, T, gpu=True):
    model.train()
    
    not_accepted_worse, accepted_worse = 0, 0
    tot = 0
    for i, data in enumerate(trainloader, 0):
        tot += 1
        inputs, labels = data
        if gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
        
        
        initial_loss = test_minibatch(inputs, labels, model)[0]
        print("Initial loss: ", initial_loss)
        # List used to keep the move to get back to the initial point
        inverse = []

        # First move
        for param in model.parameters():
            # Replicate the tensor
            tensor_size = param.data.size()
            move = torch.zeros(tensor_size)
            # Send it to the GPU
            if gpu:
                move = move.cuda()
            # Generate move
            move.normal_(std=epsilon)
            # Stepback is saved
            inverse.append(move.mul(-1)) 
            # Move the parameters
            param.data.add_(move)
        
        # Evaluate the loss 
        first_loss = test_minibatch(inputs, labels, model)[0]
        print("First loss: ", first_loss)
        
        # Second move
        for k, param in enumerate(model.parameters()):
            param.data.add_(inverse[k].mul(2))
            inverse[k] = inverse[k].mul(-1)
        second_loss = test_minibatch(inputs, labels, model)[0]
        print("Second loss: ", second_loss)

        # Get back if the first move is better
        if first_loss < second_loss:
            for k, param in enumerate(model.parameters()):
                param.data.add_(inverse[k].mul(2))
                inverse[k] = inverse[k].mul(-1)
            new_loss = first_loss
        else: new_loss = second_loss
        
        # Accept a worse solution according to temperature
        if new_loss > initial_loss and math.exp(- (new_loss - initial_loss) / T) < rm.random():
            not_accepted_worse += 1
            for k, param in enumerate(model.parameters()):
                param.data.add_(inverse[k])
            new_loss = initial_loss
        elif new_loss > initial_loss:
            accepted_worse += 1
        print("FINAL LOSS: ", test_minibatch(inputs, labels, model)[0])
        del move, inverse
        
    return not_accepted_worse, accepted_worse, tot


In [ ]:
def sgd(train_loader, model, lr):
    model.train()
    
    for i, data in enumerate(trainloader, 0):
        

In [6]:
train_loader, test_loader = ut.load_dataset(dataset_name='mnist', minibatch=512)

# Parameters
epsilon = 1e-3
epochs = 1

# Testing the second method
net = ConvNet().cuda()
training_set_measurements, validation_set_measurements, times = [], [], []
starting_time = time.time()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0):
    ut.train(train_loader, net, optimizer, criterion, 1)
    print("Training error:", ut.test_train(train_loader, net))
    print("Validation error:", ut.test(test_loader, net))
T = 1
print("?ANJDJSA")
for epoch in range(epochs):
    print("Epoch: ", epoch, ", epsilon:", epsilon)
    
    T = 0.97 * T
    not_accepted_worse, accepted_worse, tot = SGA(train_loader, net, epsilon, T)
    
    if (1 - (not_accepted_worse + accepted_worse)) / tot < 0.7:
        T = 1
    print("Non accepted: ", not_accepted_worse)
    print("Accepted: ", accepted_worse)
    
    training_set_measurements.append(ut.test_train_sample(train_loader, net))
    validation_set_measurements.append(ut.test(test_loader, net))
    
    print("Training error:", training_set_measurements[epoch])
    print("Validation error:", validation_set_measurements[epoch])


?ANJDJSA
Epoch:  0 , epsilon: 0.001
Initial loss:  0.004496290348470211
First loss:  0.004494197200983763
Second loss:  0.0044983262196183205
FINAL LOSS:  0.004494197200983763
Initial loss:  0.004498067311942577
First loss:  0.004499836824834347
Second loss:  0.004496455192565918
FINAL LOSS:  0.004496455192565918
Initial loss:  0.004498594906181097
First loss:  0.004499051719903946
Second loss:  0.004498641937971115
FINAL LOSS:  0.004498641937971115
Initial loss:  0.004492824897170067
First loss:  0.004493100102990866
Second loss:  0.004492649342864752
FINAL LOSS:  0.004492649342864752
Initial loss:  0.004502125084400177
First loss:  0.0044995457865297794
Second loss:  0.004505142569541931
FINAL LOSS:  0.004499546252191067
Initial loss:  0.004491614643484354
First loss:  0.004491531290113926
Second loss:  0.004491439089179039
FINAL LOSS:  0.004491439089179039
Initial loss:  0.00447770394384861
First loss:  0.004478149581700563
Second loss:  0.004477676935493946
FINAL LOSS:  0.004477676

FINAL LOSS:  0.0044469828717410564
Initial loss:  0.004415388684719801
First loss:  0.004414076916873455
Second loss:  0.004416964948177338
FINAL LOSS:  0.004414076916873455
Initial loss:  0.00442732498049736
First loss:  0.004422468598932028
Second loss:  0.00443268520757556
FINAL LOSS:  0.004422468598932028
Initial loss:  0.004429394844919443
First loss:  0.004428380634635687
Second loss:  0.00443120626732707
FINAL LOSS:  0.004428380634635687
Initial loss:  0.004446734208613634
First loss:  0.004450239706784487
Second loss:  0.0044434694573283195
FINAL LOSS:  0.0044434694573283195
Initial loss:  0.004436968360096216
First loss:  0.004435248672962189
Second loss:  0.004438901320099831
FINAL LOSS:  0.004435248672962189
Initial loss:  0.0044268094934523106
First loss:  0.004433284047991037
Second loss:  0.004420856479555368
FINAL LOSS:  0.004420856479555368
Initial loss:  0.0044222441501915455
First loss:  0.004423605278134346
Second loss:  0.004421385005116463
FINAL LOSS:  0.0044213850

Training error: (0.0043962231834729516, 0.21848333333333333)
Validation error: (0.004470748376846314, 0.2092)


In [ ]:
for epoch in range(200):
    print("Epoch: ", epoch, ", epsilon:", 1e-4)
    
    loss = SGA(train_loader, net, 1e-4, 0)
    print("Training error:", ut.test_train(train_loader, net))
    print("Validation error:", ut.test(test_loader, net))

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('../..'))
import src.utils as ut
net = ut.load_net(net='vgg11', dataset_name='mnist').cuda()

net.train()
print("Len: ", len(list(net.parameters())))
for param in net.parameters():
  print(param.size())
  
del net